In [15]:
import time
import requests
from bs4 import BeautifulSoup
import json
import re
import time
import faker
import pandas as pd
from tqdm import tqdm
from datetime import datetime

In [16]:
fake = faker.Faker(locale='ru')

In [17]:
yandex_headers = {
    'User-Agent': fake.chrome(),
    'accept-language': 'en-US,en;q=0.9',
    'pragma': 'np-cache',
    'content-type': 'application/json',
    'referer': 'http://yandex.ru/jobs/vacancies'
}

In [18]:
API_URL = "http://yandex.ru/jobs/api/publications/?page_size=1000&text="
VACANCY_INFO_URL = "http://yandex.ru/jobs/vacancies/"

In [19]:
# proxy = {
#     'http': 'http://130.61.236.9:80'
# }

In [20]:
clear_string = lambda x: re.sub(' +', ' ', x.replace('\n', '\n ')).strip()
title_to_url = lambda x: "-".join(re.sub('\s+', ' ', re.sub(r'[^\w\s-]', '', x.lower())).split(' '))

In [21]:
result = requests.get(API_URL, headers=yandex_headers)
result

<Response [200]>

In [22]:
all_jobs = result.json()['results']
len(all_jobs)

787

In [23]:
datetime.now()

datetime.datetime(2022, 6, 24, 7, 39, 25, 681895)

In [24]:
jobs_info = []
for job in all_jobs:
    url = VACANCY_INFO_URL + title_to_url(job['title']) + '-' + str(job['id'])
    internal_id = job['id']
    summary = job['short_summary']
    jobs_info.append((url, internal_id, job['title']))

In [25]:
yandex_jobs = []
for job in tqdm(jobs_info):
    result = requests.get(job[0], headers=yandex_headers)

    if result.status_code != 200:
        print(f'Non 200 for {job[0]}. skipping...')
        continue

    soup = BeautifulSoup(result.text)

    description_div = soup.find('div', {'class': 'lc-jobs-vacancy__section lc-jobs-vacancy__section_type_description_main'})
    description = clear_string(description_div.text) if description_div is not None else None

    responsibilities_div = soup.find('div', {'class': 'lc-jobs-vacancy__section lc-jobs-vacancy__section_type_duties'})
    responsibilities = clear_string(responsibilities_div.text) if responsibilities_div is not None else None

    qualifications_div = soup.find('div', {'class': 'lc-jobs-vacancy__section lc-jobs-vacancy__section_type_key_qualifications'})
    qualifications = clear_string(qualifications_div.text) if qualifications_div is not None else None

    if description is None and responsibilities is None and qualifications is None:
        print(f'All None for {job[0]}. skipping...')
        continue

    job_dict = {
        'title': job[2],
        'internal_id': job[1],
        'url': job[0],
        'description': description,
        'responsibilities': responsibilities,
        'qualifications': qualifications,
        'company': 'Yandex',
        'publish_date': datetime.now()
    }
    yandex_jobs.append(job_dict)

    time.sleep(0.2) # sleep for 200 ms

100%|██████████| 787/787 [24:30<00:00,  1.87s/it]  


In [26]:
snapshot = pd.DataFrame(yandex_jobs)
print(len(snapshot))
snapshot.sample(5)

787


,title,internal_id,url,description,responsibilities,qualifications,company,publish_date
262,Финансовый контролёр,9247,http://yandex.ru/jobs/vacancies/финансовый-кон...,Группа финансовых бизнес-партнёров департамент...,Что нужно делать:\n контролировать и анализиро...,"Мы ждем, что вы:\n нацелены на решение задач б...",Yandex,2022-06-24 07:47:07.662868
21,Java-разработчик в Маркет,3762,http://yandex.ru/jobs/vacancies/java-разработч...,Каждый месяц больше 20 миллионов человек выбир...,None,"Мы ждем, что вы:\n готовы к действительно слож...",Yandex,2022-06-24 07:40:06.547535
141,Разработчик бэкенда в группу потоковой обработ...,9304,http://yandex.ru/jobs/vacancies/разработчик-бэ...,Каждый день миллионы пользователей интернета в...,Что нужно делать:\n участвовать в разработке и...,"Мы ждем, что вы:\n хотите писать на C++;\n зна...",Yandex,2022-06-24 07:43:34.098759
257,Разработчик гигантских генеративных нейросетей...,5912,http://yandex.ru/jobs/vacancies/разработчик-ги...,В большинстве продуктов Яндекса работают механ...,Что нужно делать:\n проверять гипотезы о качес...,"Мы ждем, что вы:\n отлично знаете математику;\...",Yandex,2022-06-24 07:46:58.402674
583,Разработчик на Python в Yandex Pay,8896,http://yandex.ru/jobs/vacancies/разработчик-на...,Сервис Yandex Pay позволяет оплачивать покупки...,Что нужно делать:\n проектировать распределённ...,"Мы ждем, что вы:\n уверенно программируете на ...",Yandex,2022-06-24 07:57:33.727868


In [27]:
snapshot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 787 entries, 0 to 786
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   title             787 non-null    object        
 1   internal_id       787 non-null    int64         
 2   url               787 non-null    object        
 3   description       748 non-null    object        
 4   responsibilities  726 non-null    object        
 5   qualifications    787 non-null    object        
 6   company           787 non-null    object        
 7   publish_date      787 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(6)
memory usage: 49.3+ KB


In [28]:
current_date = datetime.now().strftime('%d-%m-%Y')
current_date

'24-06-2022'

In [29]:
snapshot.to_csv(f'../data/yandex/{current_date}.csv')
snapshot.to_csv(f'../data/yandex/{current_date}.tsv', sep='\t')

#### Проверка на единичной вакансии

In [195]:
result = requests.get(jobs_info[19][0], headers=yandex_headers)
result

<Response [200]>

In [196]:
result.request.url

'https://yandex.ru/jobs/vacancies/python-%D1%80%D0%B0%D0%B7%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D1%87%D0%B8%D0%BA-%D0%B2-wiki-4005'

In [197]:
soup = BeautifulSoup(result.text)

In [198]:
clear_string(soup.find('div', {'class': 'lc-jobs-vacancy__section lc-jobs-vacancy__section_type_description_main'}).text)

'Наша команда делает сервисы, которые формируют базу знаний Яндекса. Ежедневно сотрудники Яндекса обращаются за рабочей документацией, совместно редактируют статьи, делятся рабочими материалами, готовят отчеты. Работоспособность всех этих процессов обеспечивается нашими решениями.\n Исторически большинство крупных проектов написано на Python3 (Django, FastApi).'

In [199]:
clear_string(soup.find('div', {'class': 'lc-jobs-vacancy__section lc-jobs-vacancy__section_type_duties'}).text)

'Что нужно делать:\n повышать удобство работы с сервисами;\n улучшать быстродействие;\n проектировать и внедрять функциональность в ответ на новые потребности пользователей и бизнеса.'

In [200]:
clear_string(soup.find('div', {'class': 'lc-jobs-vacancy__section lc-jobs-vacancy__section_type_key_qualifications'}).text)

'Мы ждем, что вы:\n имеете опыт разработки (production-ready) и поддержки веб-сервисов. При этом важен не столько сам язык, сколько умение мыслить, писать эффективный код и обладать собственным видением;\n любите сложные и крупные сервисы;\n готовы к тесному взаимодействию с потребителями.'